In [2]:
!git log


commit cab10dc9b366db7e1067d6eb53284134df74eb23 (HEAD -> master, origin/master, origin/HEAD)
Author: Romain Jouin <romain.jouin@gmail.com>
Date:   Fri Jan 18 18:23:32 2019 +0100

    last_class

commit bec454bf8586b0e563148da86914ce7c95b17a95
Author: Romain Jouin <romain.jouin@gmail.com>
Date:   Fri Jan 11 15:34:31 2019 +0100

    tab cleaning

commit d93181566c9ef5d59370ac447b58588ad1605b7d
Author: Romain Jouin <romain.jouin@gmail.com>
Date:   Fri Jan 11 10:32:37 2019 +0100

    first

commit 5b43072a064d05c1e010bdd5de12351b5c8bf75f
Author: Romain Jouin <romain.jouin@gmail.com>
Date:   Wed Jan 9 23:44:32 2019 +0100

    usage : bokeh serve --show  bokeh_app.py

commit 2026c2b6a68a05ac4b2848084291c6648944eacf
Author: Romain Jouin <romain.jouin@gmail.com>
Date:   Fri Dec 14 07:50:18 2018 +0100

    pas mal

commit 8e13cc723025b686aca5d075f66c769820a02bd9
Merge: 323a0e31 663989b5
Author: Romain Jouin <romain.jouin@gmail.com>
Date:   Thu Dec 13 23:07:33 2018 +0100

    Merge branch 'maste

In [3]:
!git add 01_velib.ipynb

In [4]:
!git commit -m "pas mal"

[master ba28429a] pas mal
 1 file changed, 1433 insertions(+), 7200 deletions(-)
 rewrite 01_velib.ipynb (96%)


# Debut

In [5]:
def random_float(min=0, max=9):
    import random
    entier = random.randint(min, max)
    float_ = random.random()
    return entier+float_

In [6]:
class Position():
    def __init__(self, longitude, lattitude):
        self.longitude = longitude
        self.lattitude = lattitude
    def __str__(self):
        return "(lon:%.2f lat:%.2f)"%(self.longitude, self.lattitude)


# 1) Logger

In [7]:
class Logger():
    def __init__(self, path, bufferlength=1, separator="\n", mode="a"):
        import os
        if os.path.exists(path) :
            msg = "Attention : vous allez utiliser un fichier existant !"
            print (msg)
            if (mode != "a"):
                raise msg
        self.path         = path
        self.logs         = []
        self.bufferlength = bufferlength
        self.separator    = separator

    def clean_msg(self, msg):
        msg = str(msg)
        if msg[-1] != "\n": 
            msg = msg + "\n"
        return msg
    
    def log(self, msg):
        print(msg)
        msg = self.clean_msg(msg)
        import datetime
        msg=str(datetime.datetime.now().isoformat())+","+msg
        self.logs.append(msg)
        if self.bufferlength < len(self.logs):
            self.write_to_disk()
    
    def write_to_disk(self):
        with open(self.path, "a") as f:
            f.writelines(self.logs)
        self.logs= []
    
    def __del__(self):
        self.write_to_disk()
                

In [8]:
def get_random_position(x_max, y_max):
    return Position(random_float(0, y_max), random_float(0, y_max))

# 2) Velo

In [9]:
class Velo:
    nb_d_instance =0
    def __init__(self):
        import random
        Velo.nb_d_instance += 1
        self.id             = "velo_%s"%(Velo.nb_d_instance)
        self.performance    = random.random()
    def update_performance(self):
        import numpy as np
        self.performance = np.random.normal(0.75, 0.2)

# Velo - test

In [10]:
def test_velo():
    v = Velo()
    perfs = [(v.update_performance(), v.performance)[1] for x in range(10)]
    print(perfs)
test_velo()

[0.8374375271513963, 0.821214680913176, 0.8763762826976398, 0.6897402441364788, 0.6308387640471866, 0.9112882899626995, 0.7492201854391028, 0.5873196300959094, 0.6268716872541812, 0.6891882275750865]


# 3) Trajet

In [11]:
p1 = Position(0,0)

In [12]:
p2 = Position(1,1)

In [13]:
v = 1
t = Trajet(p1,p2, v)

NameError: name 'Trajet' is not defined

In [14]:
[print(t.position, t.next_position(), t.position) for i in range(11)]

NameError: name 't' is not defined

In [15]:
class Trajet():
    def __init__(self, position_depart, position_arrivee, vitesse=3):
        self.depart      = position_depart
        self.destination = position_arrivee
        self.nb_step     = min(10, 10/vitesse)
        self.set_position(position_depart)
        self.set_distance() 
        self.set_pas()
        self.avancee_minimale = 0.1
    def __str__(self):
        return "%s -> %s"%(self.depart, self.destination)
        
    def set_position(self, position):
        self.longitude   = position.longitude
        self.lattitude   = position.lattitude
        self.position    = position
        
    def set_pas(self):
        """
        @Todo : ici le point d'arrivée est toujours atteint en 3 temps.
        Il faudrait le rendre dépendant de la distance.
        => rajouter un paramètre de vitesse ? 
        """
        nb_step            = self.nb_step
        delta_longitude    = self.destination.longitude  - self.longitude
        delta_lattitude    = self.destination.lattitude  - self.lattitude
        
        self.pas_longitude = delta_longitude * 1/ nb_step
        self.pas_lattitude = delta_lattitude * 1/ nb_step
        
    def set_distance(self):
        import math
        delta_longitude = self.destination.longitude  - self.longitude
        delta_lattitude = self.destination.lattitude  - self.lattitude
        distance        = math.sqrt(math.pow(delta_longitude, 2) + math.pow(delta_lattitude, 2))
        self.distance   = distance
        return distance
        
    def next_position(self):
        last_distance = self.set_distance() 
        if self.distance < self.avancee_minimale:
            self.set_position(self.destination)
            return self.destination
        else:
            new_longitude   = self.longitude + self.pas_longitude
            new_latitude    = self.lattitude + self.pas_lattitude
            new_position    = Position(new_longitude, new_latitude)
            self.set_position(new_position)
            new_distance    = self.set_distance()
            if not new_distance<last_distance:
                new_position = self.destination
                self.set_position(new_position)
            return new_position

In [16]:
def test_trajet():
    # de 0,0 a 0,0
    depart = Position(longitude =0,lattitude=0)
    arrivee = Position(longitude=0,lattitude=0)
    t = Trajet(depart, arrivee)
    assert (t.depart==depart)
    assert (t.destination==arrivee)
    assert (t.distance==0)
    print(t.pas_longitude)
    print(t.pas_lattitude)
    assert (t.pas_longitude==0)
    assert (t.pas_lattitude==0)
    print([(t.next_position(), "%s"%t.position)[1] for x in range(4)])
    
    # de 0,0 a 0,1
    depart = Position(longitude =0,lattitude=0)
    arrivee = Position(longitude=0,lattitude=1)
    t = Trajet(depart, arrivee)
    assert (t.depart==depart)
    assert (t.destination==arrivee)
    assert (t.distance==1)
    print(t.pas_longitude)
    print(t.pas_lattitude)
    assert (t.pas_longitude==0)
    assert (t.pas_lattitude==1/3)
    print([(t.next_position(), "%s"%t.position)[1] for x in range(4)])
    
    # de 0,0 a 1,1
    depart = Position(longitude =0,lattitude=0)
    arrivee = Position(longitude=1,lattitude=1)
    t = Trajet(depart, arrivee)
    assert (t.depart==depart)
    assert (t.destination==arrivee)
    import math
    assert (t.distance==math.sqrt(2))
    print(t.pas_longitude)
    print(t.pas_lattitude)
    assert (t.pas_longitude==1/3)
    assert (t.pas_lattitude==1/3)
    print([(t.next_position(), "%s"%t.position)[1] for x in range(4)])
    
    # de 0,0 a 1,2
    depart = Position(longitude =0,lattitude=0)
    arrivee = Position(longitude=1,lattitude=2)
    t = Trajet(depart, arrivee)
    assert (t.depart==depart)
    assert (t.destination==arrivee)
    import math
    assert (t.distance==math.sqrt(2*2+1*1))
    print(t.pas_longitude)
    print(t.pas_lattitude)
    assert (t.pas_longitude==1/3)
    assert (t.pas_lattitude==2/3)
    print([(t.next_position(), "%s"%t.position)[1] for x in range(4)])
test_trajet()
    

0.0
0.0
['(lon:0.00 lat:0.00)', '(lon:0.00 lat:0.00)', '(lon:0.00 lat:0.00)', '(lon:0.00 lat:0.00)']
0.0
0.3


AssertionError: 

# 4) Station

In [17]:
class Station:
    nb_d_instance =0
    station_logger=Logger("./stations.csv")
    def __init__(self, position):
        import random
        Station.nb_d_instance +=1
        NB_BORNES_POSSIBLES = [5,10,15,20,30]
        self.position       = position
        self.nb_bornes      = random.choice(NB_BORNES_POSSIBLES)
        self.velos          = [Velo() for i in range(self.nb_bornes)]
        self.id             = Station.nb_d_instance
    def __str__(self):
        to_print = "Station n° {nom} : {position}, {nb_bornes} bornes, {nb_velos_dispo} velos"
        
        return to_print.format(nom=self.id,
                               position=self.position,
                              nb_bornes=self.nb_bornes,
                              nb_velos_dispo=len(self.velos))
    def nb_de_velos(self):
        return len(self.velos)

    def has_empty_place(self):
        return len(self.velos) < self.nb_bornes

    def has_velo(self):
        return 0 < len(self.velos)
    def log(self, message):
        message="station_%s : %s"%(self.id, message)
        Station.station_logger.log(message)
        
    def deposer_un_velo(self, velo):
        if self.has_empty_place():
            velo.update_performance()
            self.velos.append(velo)
            self.log("recupere le velo %s"%velo.id)
            return True
        else:
            self.log("complete")
            return False
    def prendre_un_velo(self):
        if self.has_velo():
            velo = self.velos.pop()
            self.log("donne le velo %s"%velo.id)
            return velo
        else:
            self.log("vide.")
            return False

    def nb_velo_under_perf(self, seuil_de_performance=0.5):
        test = lambda velo: velo.performance < seuil_de_performance
        return len(list(filter(test, self.velos)))
        

Attention : vous allez utiliser un fichier existant !


# 5) Prestataire

In [18]:
class Prestataire():
    def __init__(self, largeur=10, hauteur=10):
        self.largeur = largeur
        self.hauteur = hauteur
        self.reseau = []
        for lattitude in range(self.hauteur):
            for longitude in range(self.largeur):
                emplacement = Position(lattitude, longitude)
                new_station = Station(emplacement)
                self.reseau.append(new_station)
                
    def get_closest_station(self, position):
        """
        @Todo : améliorer pour les valeurs en dehors du réseau
        """
        nb_ligne = round(position.longitude)
        nb_col   = round(position.lattitude)
        indice   =  (nb_ligne * self.largeur) + (nb_col )
        if nb_ligne<0 or nb_col<0:
            closest  = self.reseau[0]    
        try:
            closest  = self.reseau[int(indice)]
            if is_same_position(closest, position):
                print("same position")
                closest = self.reseau[int(indice)+1]
        except:
            closest  = self.reseau[-1]
        return closest
    

    def filtrer_le_reseau(self, function_de_test):
        """
        Renvoie les stations qui passe le test.
        
        Parameters:
            function de test : lambda function
        Return:
            list
        
        """
        return list(filter(function_de_test, self.reseau))
    def nb_station_passant_le_test(self, function):
        r = self.filtrer_le_reseau(function)
        return len(r)
    
    def nombre_de_stations(self):
        return len(self.reseau)
    
    def nombre_de_velos(self):
        return sum(map(lambda x: x.nb_de_velos(),self.reseau))
    
    def nombre_de_stations_pleine(self):
        test  = lambda station: station.nb_bornes == len(station.velos)
        return self.nb_station_passant_le_test(test)
    
    def nombre_de_stations_avec_velo(self):
        test = lambda x: x.has_velo()
        return self.nb_station_passant_le_test(test)
    
    def nombre_de_stations_vide(self):
        test  = lambda station: not station.has_velo()
        return self.nb_station_passant_le_test(test)

    def __str__(self):
        return """
                Nb de stations           : %s 
                Nb de vélos              : %s 
                Nb de stations avec vélo : %s 
                Nb de stations pleines   : %s
                Nb de stations vides     : %s
        """%(self.nombre_de_stations(),
             self.nombre_de_velos(),
             self.nombre_de_stations_avec_velo(),
             self.nombre_de_stations_pleine(),
             self.nombre_de_stations_vide())
    
    def nombre_de_velos_sous_performants(self, seuil=0.5):
        r = map(lambda station: station.nb_velo_under_perf(seuil), self.reseau)
        return sum(list(r))
    def get_topology(self):
        """
        Return a Datframe of longitude/lattitude
        """
        import pandas as pd
        infos = { "longitudes" : [x.position.longitude for x in self.reseau],
                  "lattitude"  : [x.position.lattitude for x in self.reseau]}
        
        return pd.DataFrame(infos)
        
        
    

# 4.1) Test decaux

In [19]:
def test_prestataire():
    def compare_positions(p1,p2):
        test1 = p1.longitude==p2.longitude 
        test2 = p1.lattitude==p2.lattitude 
        return test1 and test2    
    p = Prestataire(largeur=10, hauteur=10)
    point_minusOne_minusOne = Position(-1,-1)
    point_zero_zero = Position(0,0)
    point_un_un     = Position(1,1)
    point_zero5_zero5     = Position(0.5,0.5)
    point_zero6_zero6     = Position(0.6,0.6)
    point_2_4_3_2     = Position(longitude=2.4,lattitude=3.2)
    point_2_3     = Position(longitude=2,lattitude=3)
    point_9_9     = Position(longitude=9,lattitude=9)
    point_10_10     = Position(longitude=10,lattitude=10)
    print(p.get_closest_station(point_zero_zero).position==Position(0,0))
    assert(compare_positions(point_zero_zero, p.get_closest_station(point_zero_zero).position))
    assert(compare_positions(point_un_un, p.get_closest_station(point_un_un).position))
    assert(compare_positions(point_zero_zero, p.get_closest_station(point_zero5_zero5).position   ))
    assert(compare_positions(point_un_un, p.get_closest_station(point_zero6_zero6).position   ))
    assert(compare_positions(point_2_3, p.get_closest_station(point_2_4_3_2).position       ))
    print(p.get_closest_station(point_10_10).position)
    assert(compare_positions(point_9_9, p.get_closest_station(point_10_10).position       ))    
    assert(compare_positions(point_zero_zero, p.get_closest_station(point_minusOne_minusOne).position       ))        
test_prestataire()

False


AssertionError: 

In [20]:
def draw(positions):
    import pandas as pd
    df = pd.DataFrame([ {"lon" : x.longitude, 
                "lat": x.lattitude} for x in positions])
    df.plot.scatter(x="lon", y="lat")
    
    

In [21]:
r = Prestataire(3,3)

# 6) Villes

In [22]:
class Ville:
    def __init__(self, nb_de_cyclistes, Prestataire):
        self.prestataire = Prestataire
        x_max = Prestataire.largeur -2
        y_max = Prestataire.hauteur-2
        self.cyclistes   = [Cycliste(get_random_position(x_max, y_max), get_random_position(x_max, y_max), self) for i in range(nb_de_cyclistes)]
        self.ax = None
        
    def reveil(self):
        list(map(lambda cycliste : cycliste.avancer(), self.cyclistes))
        
    def get_cycliste_workplaces(self):
        import pandas as pd
        infos = { "longitudes" : [cycliste.travail.longitude for cycliste in self.cyclistes],
                  "lattitude"  : [cycliste.travail.lattitude for cycliste in self.cyclistes]}
        return pd.DataFrame(infos)

    def get_cycliste_homes(self):
        import pandas as pd
        infos = { "longitudes" : [cycliste.home.longitude for cycliste in self.cyclistes],
                  "lattitude"  : [cycliste.home.lattitude for cycliste in self.cyclistes]}
        return pd.DataFrame(infos)

        
    def get_cycliste_positions(self):
        import pandas as pd
        infos = { "longitudes" : [x.position.longitude for x in self.cyclistes],
                  "lattitude"  : [x.position.lattitude for x in self.cyclistes]}
        return pd.DataFrame(infos)

    def get_topology(self):
        import pandas as pd
        infos = { "longitudes" : [x.position.longitude for x in self.prestataire.reseau],
                  "lattitude"  : [x.position.lattitude for x in self.prestataire.reseau]}
        return pd.DataFrame(infos)
    def get_drawing_data(self):
        cyclistes  = self.get_cycliste_positions()
        stations   = self.get_topology()
        homes      = self.get_cycliste_homes()
        workplaces = self.get_cycliste_workplaces()
        return cyclistes, stations, homes, workplaces

    def draw_map(self,ax=None):
        import matplotlib.pyplot as plt
        cyclistes, stations, homes, workplaces = self.get_drawing_data()
        if not ax:
            ax = stations.plot.scatter(      x="longitudes", y="lattitude", c="grey"   , marker="D", alpha=0.4)
        else:
            stations.plot.scatter(      x="longitudes", y="lattitude", c="grey"   , marker="D", alpha=0.1, ax=ax)
        homes.plot.scatter(         x="longitudes", y="lattitude", c="green"  , marker="s", ax=ax )
        workplaces.plot.scatter(    x="longitudes", y="lattitude", c="red"    , marker="x", ax=ax )
        cyclistes.plot.scatter(x="longitudes", y="lattitude", c="blue", marker="1"        , ax=ax)
        fig = plt.gcf()
        fig.suptitle(str(cyclistes))
        return ax
        

In [23]:
def test_ville():
    
    p = Prestataire(3,3)
    nb_cyclistes = 2
    v = Ville(nb_cyclistes,p)
    assert(len(v.cyclistes)==nb_cyclistes)

# 7) Cycliste

In [24]:
Cycliste(p1,p2, paris)

NameError: name 'Cycliste' is not defined

In [25]:
class Cycliste:
    nb_d_instance =0
    cycliste_logger= Logger("./cyclistes.csv")
    def __init__(self, position_home, position_travail, ville):
        import random
        import numpy as np
        Cycliste.nb_d_instance +=1
        self.id           = Cycliste.nb_d_instance
        self.home         = position_home
        self.travail      = position_travail
        self.est_en_route = False
        self.position     = position_home
        self.destination  = position_travail
        self.sur_velo     = False
        self.velo         = False
        self.sexe         = random.choice(["H", "F"])
        self.age          = random.choice(range(15,85))
        self.sportivite   = np.random.normal(2, 2)
        self.vitesse_a_pied = 2 * self.sportivite 
        self.vitesse_a_velo = 5 * self.sportivite
        self.ville        = ville
        self.set_next_trajet()
    
    def vitesse(self):
        if self.sur_velo:
            return self.vitesse_a_velo
        else:
            return self.vitesse_a_pied
        
    def est_a_la_maison(self):
        return self.position == self.home
    def est_au_travail(self):
        return self.position == self.travail
    def est_arrive(self):
        return self.position == self.destination
    
    def log(self, message):
        message="cycliste_%s : %s"%(self.id, message)
        Cycliste.cycliste_logger.log(message)
        
    def trouve_velo(self):
        limite = 10
        while not self.sur_velo and 0 < limite :
            limite         = limite - 1
            station_proche = self.ville.prestataire.get_closest_station(self.position)
            self.log("Go to station : %s %s"%(station_proche.id, station_proche.position))
            self.go_to(station_proche.position)
            if self.est_arrive():
                self.log("essaie station : %s "%station_proche.id)
                velo           = station_proche.prendre_un_velo()
                self.position  = station_proche.position 
                self.destination = self.destination_finale
                if velo:
                    self.log("trouve velo %s sur station %s"%(velo.id, station_proche.id))
                    self.sur_velo = True # break the loop
                    self.velo     = velo
                    
                    self.set_next_trajet()
                
                
    def rendre_velo(self):
        limite = 10
        while self.sur_velo and 0 < limite :
            limite         = limite - 1
            station_proche = self.ville.prestataire.get_closest_station(self.position)
            velo_rendu     = station_proche.deposer_un_velo(self.velo)
            self.log("essaie de rendre sur la station : %s "%station_proche.id)
            self.position  = station_proche.position 
            if velo_rendu:
                self.log( "velo rendu sur la station sur : %s"%station_proche.id)
                self.sur_velo = False
                self.velo     = False
                self.set_next_trajet()
                
    def set_next_trajet(self):
        if self.est_a_la_maison():
            self.log("next trajet = travail")
            self.destination = self.travail
            self.destination_finale = self.travail
        if self.est_au_travail():
            self.log("next trajet = maison")
            self.destination = self.home
            self.destination_finale = self.home
        self.trajet = Trajet(self.position, self.destination, self.vitesse())
    
    def ou_suis_je(self):
        if self.est_a_la_maison():
            return " a la maison"
        if self.est_au_travail():
            return " au travail"
        if self.est_en_route:
            return " en route"
    def ou_vais_je(self):
        if self.destination == self.travail:
            return " au travail"
        if self.destination == self.home:
            return " a la maison"
    def go_to(self,position):
        self.destination = position
        self.trajet      = Trajet(self.position, self.destination)
        print("current = %s, next =%s, est_arrive=%s, trajet=%s"%(self.position, self.trajet.next_position(), self.est_arrive(), self.trajet)) 
        self.position    = self.trajet.next_position()
        
    def avancer(self):
        if self.sur_velo:
            self.log( "est sur un velo")
            if self.est_arrive():
                self.log( "est arrive %s(%s)"%(self.ou_suis_je(), self.position.__dict__))
                self.set_next_trajet()
                self.rendre_velo()
            else:
                self.go_to(self.destination)
                self.log( "va a la prochaine position (%s): %s "%(self.ou_vais_je(), self.position.__dict__))
        else:
            self.log( "va chercher un velo")
            self.trouve_velo()

Attention : vous allez utiliser un fichier existant !


# 7) Algo

In [26]:
%matplotlib
JcDecaux = Prestataire(10,10)
print(JcDecaux)
paris = Ville(1, JcDecaux)
paris.draw_map()

Using matplotlib backend: MacOSX

                Nb de stations           : 100 
                Nb de vélos              : 1635 
                Nb de stations avec vélo : 100 
                Nb de stations pleines   : 100
                Nb de stations vides     : 0
        
cycliste_1 : next trajet = travail


In [27]:
def is_same_position(p1, p2):
    return p1.lattitude==p2.lattitude and p1.longitude==p2.longitude

In [28]:
import pandas as pd

In [29]:
ax = False
positions = pd.DataFrame()
for i in range(50):
    paris.reveil()
    paris.draw_map()
    positions = positions.append(paris.get_cycliste_positions())
    print("-"*30)

cycliste_1 : va chercher un velo
cycliste_1 : Go to station : 209 (lon:9.00 lat:9.00)
current = (lon:6.40 lat:3.86), next =(lon:7.18 lat:5.40), est_arrive=False, trajet=(lon:6.40 lat:3.86) -> (lon:9.00 lat:9.00)
cycliste_1 : Go to station : 209 (lon:9.00 lat:9.00)
current = (lon:7.96 lat:6.94), next =(lon:8.27 lat:7.56), est_arrive=False, trajet=(lon:7.96 lat:6.94) -> (lon:9.00 lat:9.00)
cycliste_1 : Go to station : 209 (lon:9.00 lat:9.00)
current = (lon:8.58 lat:8.18), next =(lon:8.71 lat:8.42), est_arrive=False, trajet=(lon:8.58 lat:8.18) -> (lon:9.00 lat:9.00)
cycliste_1 : Go to station : 209 (lon:9.00 lat:9.00)
current = (lon:8.83 lat:8.67), next =(lon:8.88 lat:8.77), est_arrive=False, trajet=(lon:8.83 lat:8.67) -> (lon:9.00 lat:9.00)
cycliste_1 : Go to station : 209 (lon:9.00 lat:9.00)
current = (lon:8.93 lat:8.87), next =(lon:8.95 lat:8.91), est_arrive=False, trajet=(lon:8.93 lat:8.87) -> (lon:9.00 lat:9.00)
cycliste_1 : Go to station : 209 (lon:9.00 lat:9.00)
current = (lon:8.97

/Users/romain/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


------------------------------
cycliste_1 : est sur un velo
current = (lon:0.79 lat:0.45), next =(lon:0.73 lat:0.38), est_arrive=False, trajet=(lon:0.79 lat:0.45) -> (lon:0.57 lat:0.22)
cycliste_1 : va a la prochaine position ( au travail): {'longitude': 0.6607805310572554, 'lattitude': 0.3130942561430805} 
------------------------------
cycliste_1 : est sur un velo
current = (lon:0.66 lat:0.31), next =(lon:0.63 lat:0.29), est_arrive=False, trajet=(lon:0.66 lat:0.31) -> (lon:0.57 lat:0.22)
cycliste_1 : va a la prochaine position ( au travail): {'longitude': 0.5745034463478574, 'lattitude': 0.22322002924252382} 
------------------------------
cycliste_1 : est sur un velo
cycliste_1 : est arrive  au travail({'longitude': 0.5745034463478574, 'lattitude': 0.22322002924252382})
cycliste_1 : next trajet = maison
station_209 : recupere le velo velo_3371
cycliste_1 : essaie de rendre sur la station : 209 
cycliste_1 : velo rendu sur la station sur : 209
------------------------------
cycliste_

In [30]:
positions

,longitudes,lattitude
0,9.000000,9.000000
0,3.944702,3.733932
0,1.922583,1.627505
0,1.113735,0.784934
0,0.790196,0.447906
0,0.660781,0.313094
0,0.574503,0.223220
0,9.000000,9.000000
0,9.000000,9.000000
0,7.437478,5.916410


In [31]:
!cat stations.csv

2018-12-14T07:50:18.712587,station_111 : donne le velo velo_1701
2018-12-14T07:50:18.724545,station_111 : recupere le velo velo_1701
2018-12-14T07:50:19.659138,station_124 : donne le velo velo_1871
2018-12-14T07:50:19.662202,station_124 : donne le velo velo_1870
2018-12-14T07:50:19.718306,station_124 : recupere le velo velo_1871
2018-12-14T07:50:19.719815,station_124 : recupere le velo velo_1870
2018-12-14T07:50:19.775006,station_124 : donne le velo velo_1870
2018-12-14T07:50:19.776565,station_124 : donne le velo velo_1871
2018-12-14T07:50:19.829352,station_124 : recupere le velo velo_1870
2018-12-14T07:50:19.830937,station_124 : recupere le velo velo_1871
2018-12-14T07:50:19.885018,station_124 : donne le velo velo_1871
2018-12-14T07:50:19.886617,station_124 : donne le velo velo_1870
2018-12-14T07:50:19.936857,station_124 : recupere le velo velo_1871
2018-12-14T07:50:19.938273,station_124 : recupere le velo velo_1870
2018-12-14T07:50:19.996856,station_124 : donne le velo velo_1870
2018

In [ ]:
if False:
    path_to_cyclistes = "./cyclistes.csv"
    logger_ = Logger(path_to_cyclistes)
    infos_voulues = ["id", "sexe", "sportivite", "age", "sur_velo"]
    to_write      = ",".join(infos_voulues)
    logger_.log(to_write)
    for cycliste in paris.cyclistes:
        dico          = cycliste.__dict__        
        infos         = [str(dico[voulue]) for voulue in infos_voulues]
        to_write      = ",".join(infos)
        logger_.log(to_write)


# Exemple de dataFrame pandas

In [ ]:
logger_.write_to_disk()


In [31]:
import pandas as pd

In [32]:
dico = {}
for i in range(12):
    seuil       = float(i/10)
    nb_velo     = test_reseau.nombre_de_velos_sous_performants(seuil)
    dico[seuil] = nb_velo

NameError: name 'test_reseau' is not defined

In [ ]:
%matplotlib inline
pd.DataFrame(dico, index=range(12)).T[0].plot(kind='bar')